# Product Quantization for Similarity Search

## How to compress and fit a humongous set of vectors in memory for similarity search with asymmetric distance computation (ADC)

### [Click here to read and learn how Product Quantization works (with detailed explanation and illustrations)](https://peggy1502.medium.com/2f1f67c5fddd)


In [1]:
import numpy as np
from scipy.cluster.vq import kmeans2, vq
from scipy.spatial.distance import cdist

- M = number of segments
- k = number of centroids per segment
- s = dimension, or length of a segment

In [2]:
def PQ_train(vectors, M, k):
    s = int(vectors.shape[1] / M)                      # dimension (or length) of a segment.
    codebook = np.empty((M, k, s), np.float32)         
        
    for m in range(M):
        sub_vectors = vectors[:, m*s:(m+1)*s]          # sub-vectors for segment m.
        codebook[m], label = kmeans2(sub_vectors, k)   # run k-means clustering for each segment.
        
    return codebook        

In [3]:
def PQ_encode(vectors, codebook):
    M, k, s = codebook.shape
    PQ_code = np.empty((vectors.shape[0], M), np.uint8)
    
    for m in range(M):
        sub_vectors = vectors[:, m*s:(m+1)*s]           # sub-vectors for segment m.
        centroid_ids, _ = vq(sub_vectors, codebook[m])  # vq returns the nearest centroid Ids.
        PQ_code[:, m] = centroid_ids                    # assign centroid Ids to PQ_code.
        
    return PQ_code

In [4]:
def PQ_search(query_vector, codebook, PQ_code):
    M, k, s = codebook.shape
    #=====================================================================
    # Build the distance table.
    #=====================================================================
    
    distance_table = np.empty((M, k), np.float32)    # Shape is (M, k)    
        
    for m in range(M):
        query_segment = query_vector[m*s:(m+1)*s]    # query vector for segment m.
        distance_table[m] = cdist([query_segment], codebook[m], "sqeuclidean")[0]
        
    #=====================================================================
    # Look up the partial distances from the distance table.
    #=====================================================================
    
    N, M = PQ_code.shape
    distance_table = distance_table.T               # Transpose the distance table to shape (k, M)
    distances = np.zeros((N, )).astype(np.float32)

    for n in range(N):                              # For each PQ Code, lookup the partial distances.
        for m in range(M):
            distances[n] += distance_table[PQ_code[n][m]][m] # Sum the partial distances from all the segments.
            
    return distance_table, distances    

In [5]:
# def PQ_search(query_vector, codebook, PQ_code):
#     M, k, s = codebook.shape
#     distance_table = np.empty((M, k), np.float32)              # Shape is (M, k)   
    
#     for m in range(M):
#         query_segment = query_vector[m*s:(m+1)*s]               # query vector for segment m.
        
#         distance_table[m] = cdist([query_segment], codebook[m], "sqeuclidean")[0]
# #       distance_table[m] = np.linalg.norm(codebook[m] - query_segment, axis=1) ** 2
        
#     distances = np.sum(distance_table[range(M), PQ_code], axis=1)
    
#     return distances    

# Test Case 1

#### A small example with 10 database vectors (of length 6) that will be divided and splitted into 2 segments, with 4 centroids per segment.

#### You may use this example to verify and inspect the values for better understanding of Product Quantization.

In [6]:
M = 2
k = 4
vector_dim = 6          # Dimension of a vector
total_vectors = 10

np.random.seed(2022)
vectors = np.random.random((total_vectors, vector_dim)).astype(np.float32)   # Database vectors
q = np.random.random((vector_dim, )).astype(np.float32)                      # Query vector

codebook = PQ_train(vectors, M, k)
PQ_code = PQ_encode(vectors, codebook)
distance_table, distances = PQ_search(q, codebook, PQ_code)

In [7]:
vectors   # database vectors

In [8]:
q                 # query vector

In [9]:
codebook          # the generated codebook

In [10]:
PQ_code           # the generated PQ codes

In [11]:
distance_table    # the generated distance table

In [12]:
distances # the distances between the database vectors and the query vector.

# Test Case 2
#### An example with 1 million database vectors (of length 128) that will be divided and splitted into 8 segments, with 256 centroids per segment.

In [13]:
M = 8
k = 256
vector_dim = 128          # Dimension (length) of a vector
total_vectors = 1000000

np.random.seed(2022)
vectors = np.random.random((total_vectors, vector_dim)).astype(np.float32)   # Database vectors
q = np.random.random((vector_dim, )).astype(np.float32)                      # Query vector

codebook = PQ_train(vectors, M, k)
PQ_code = PQ_encode(vectors, codebook)
distance_table, distances = PQ_search(q, codebook, PQ_code)

In [14]:
vectors            # database vectors

In [15]:
q                 # query vector

In [16]:
codebook          # the generated codebook

In [17]:
PQ_code          # the generated PQ codes

In [18]:
distance_table    # the generated distance table

In [19]:
distances        # the distances between the database vectors and the query vector.